In [2]:
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # for reproducibility

In [3]:
# Predicting animal type based on various features
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)

(101, 16) (101, 1)


In [4]:
nb_classes = 7  # 0 ~ 6

In [5]:
X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6
Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot
print("one_hot", Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print("reshape", Y_one_hot)

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')


one_hot Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)
reshape Tensor("Reshape:0", shape=(?, 7), dtype=float32)


In [6]:
# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                 labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

In [7]:
prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [8]:
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2000):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 100 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={
                                 X: x_data, Y: y_data})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(
                step, loss, acc))

Step:     0	Loss: 6.310	Acc: 8.91%
Step:   100	Loss: 0.442	Acc: 81.19%
Step:   200	Loss: 0.287	Acc: 83.17%
Step:   300	Loss: 0.217	Acc: 85.15%
Step:   400	Loss: 0.175	Acc: 86.14%
Step:   500	Loss: 0.147	Acc: 88.12%
Step:   600	Loss: 0.126	Acc: 88.12%
Step:   700	Loss: 0.111	Acc: 90.10%
Step:   800	Loss: 0.100	Acc: 90.10%
Step:   900	Loss: 0.090	Acc: 90.10%
Step:  1000	Loss: 0.083	Acc: 88.12%
Step:  1100	Loss: 0.076	Acc: 88.12%
Step:  1200	Loss: 0.071	Acc: 88.12%
Step:  1300	Loss: 0.066	Acc: 89.11%
Step:  1400	Loss: 0.063	Acc: 89.11%
Step:  1500	Loss: 0.059	Acc: 89.11%
Step:  1600	Loss: 0.056	Acc: 89.11%
Step:  1700	Loss: 0.054	Acc: 90.10%
Step:  1800	Loss: 0.051	Acc: 90.10%
Step:  1900	Loss: 0.049	Acc: 90.10%


In [9]:
    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

RuntimeError: Attempted to use a closed Session.